Лабораторна робота № 7
Смислов Даніл
ІП-01

Спочатку імпортуємо потрібні для роботи бібліотеки

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

Тепер давайте імпортуємо датасет з інформацією про захворювання на Covid.

In [ ]:
def getDataset():
    data = pd.read_csv('covidInfo.csv',index_col=['date'],parse_dates=['date'])
    return data 

dataset = getDataset()
dataset.info()
dataset.head(10)

Тепер давайте залишимо саме потрібні нам дані.

In [ ]:
dataset = dataset[['location','new_cases']]
dataset.info()
dataset.tail(10)

Для зручності давайте перейменуємо стовпчики

In [ ]:
dataset = dataset.rename(columns={'location':'Country','new_cases':'New Cases'})
dataset.info()
dataset.tail(10)

Тепер давайте згрупуємо дані по країнах та оберемо для України та межуючої з нею Польщі.

In [ ]:
dataset = dataset[(dataset['Country']=='Ukraine') | (dataset['Country']=='Poland')]
print(dataset)

Давайте перевіримо на коректність даних.

In [ ]:
print(dataset[dataset['New Cases']<0])

Добре, давайте переробимо в таблицю, з якою буде зручніше працювати.

In [ ]:
dataset = dataset.pivot_table(values = ['New Cases'],index = ['date'],columns = ['Country'])

dataset.info()
dataset.head(100)

Гуд, можемо працювати з даними. Давайте візуалізуємо динаміку Covid в часі в Польщі та Україні.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
dataset['New Cases'][['Ukraine','Poland']].plot(ax=ax)
plt.title('Динаміка Covid в часі в Польщі та Україні.')
ax.grid()
plt.show()

Давайте проаналізуємо окремо дані про захворювання в Україні.

In [ ]:
ukrDataset = dataset['New Cases']['Ukraine']
ukrDataset.describe()


Тепер про захворювання в Польщі.

In [ ]:
polDataset = dataset['New Cases']['Poland']
polDataset.describe()

Тепер давайте побудуємо 2 гістограми по кількості захворювань для країн.

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (12,8))
ax[0].set_title('Ukraine')
ax[0].hist(ukrDataset)
ax[1].set_title('Poland')
ax[1].hist(polDataset)

plt.show()

Для кращої візуалізації властивостей ряду давайте застосуємо згладжування за допомогою ковзаючого середнього.

In [ ]:
def plot_moving_average(series,n):
    rolling_mean = series.rolling(window=n).mean()
    plt.figure(figsize=(15,5))
    plt.title(f'Moving average \n window size = {n}')
    plt.plot(rolling_mean,c = 'orange',label = 'Rolling mean trend')
    plt.plot(series[n:],label = 'Actual values')
    plt.legend(loc = 'upper left')
    plt.grid(True)
    plt.show()

print("Ukraine")
plot_moving_average(ukrDataset,5)
plot_moving_average(ukrDataset,10)
plot_moving_average(ukrDataset,20)
print("Poland")
plot_moving_average(polDataset,5)
plot_moving_average(polDataset,10)
plot_moving_average(polDataset,20)

Тепер давайте візуалізуємо декомпозицію ряду на тренд, сезонність та залишки для даних про захворювання в Україні.

In [ ]:
import statsmodels.tsa.api as smt

decomposition = smt.seasonal_decompose(ukrDataset[~ukrDataset.isna()])

fig = decomposition.plot()
fig.set_size_inches(15,10)
plt.show()


Тепер для даних про захворювання в Польщі.

In [ ]:
decomposition = smt.seasonal_decompose(polDataset[~polDataset.isna()])

fig = decomposition.plot()
fig.set_size_inches(15,10)
plt.show()


Тепер давайте побудуємо графіки автокореляції та часткової автокореляції.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2,figsize=(15,10))
fig.suptitle('Ukraine',fontsize = 30)
ax[0] = plot_acf(ukrDataset[~ukrDataset.isna()],ax=ax[0],lags = 120)
ax[1] = plot_pacf(ukrDataset[~ukrDataset.isna()],ax=ax[1],lags = 120)

fig, ax = plt.subplots(2,figsize=(15,10))
fig.suptitle('Poland',fontsize = 30)
ax[0] = plot_acf(polDataset[~polDataset.isna()],ax=ax[0],lags = 120)
ax[1] = plot_pacf(polDataset[~polDataset.isna()],ax=ax[1],lags = 120)

Тепер перевіримо ряд на стаціонарність за допомогою доповненого тесту Дікі-Фуллера.

In [ ]:
def dickey_fuller_test(series):
    test = smt.adfuller(series,autolag = 'AIC')
    print('adf: ',test[0])
    print('p-value: ',test[1])
    print('Critical values: ',test[4])
    if(test[0]>test[4]['5%']):
        print('Наявні одиничні корені, ряд не стаціонарний')
    else:
        print('Одиничні корені відсутні, ряд стаціонарний.')

dickey_fuller_test(ukrDataset[~ukrDataset.isna()])
dickey_fuller_test(polDataset[~polDataset.isna()])

Тепер давайте виконаємо другу частину основного завдання. Для початку імпортуємо датасет та відсортуємо за датою.

In [ ]:
def getDataset():
    data = pd.read_csv('USD_UAH.csv',index_col=['Date'],parse_dates=['Date'])
    return data 


dataset = getDataset()
dataset = dataset.sort_index(ascending=True)
dataset.info()
dataset.head(10)

Як ми бачимо, є пропуски в датах, тому давайте це виправимо.

In [ ]:

idx = pd.date_range(dataset.index[0], dataset.index[-1])
dataset = dataset.reindex(idx)
print(dataset.head(10))

for index in dataset.index:
        if pd.isna(dataset.loc[index, 'Price']):
            dataset.loc[index] = dataset.shift().loc[index]
            
print(dataset.head(10))


Тепер перевіримо дані на коректність.

In [ ]:
print(dataset[dataset['Price']<0])
print(dataset[dataset['Open']<0])
print(dataset[dataset['High']<0])
print(dataset[dataset['Low']<0])

Як бачимо, від'ємних даних немає. Можемо працювати з даними. Давайте візуалізуємо динаміку ціни по датах.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
dataset['Price'].plot(ax=ax,subplots=True)
ax.grid(True)
plt.show()

Тепер використаємо згладжування для дослідження характеристик ряду.

In [ ]:
plot_moving_average(dataset['Price'],10)
plot_moving_average(dataset['Price'],20)
plot_moving_average(dataset['Price'],30)

Давайте декомпозуємо ціну на тренд, сезонність та залишки.

In [ ]:
decomposition = smt.seasonal_decompose(dataset['Price'])
fig = decomposition.plot()
fig.set_size_inches(15,10)
plt.show()

Тепер побудуємо графіки автокореляції та часткової автокореляції.

In [ ]:
fig, ax = plt.subplots(2,figsize=(15,10))
ax[0] = plot_acf(dataset['Price'], ax=ax[0], lags=100)
ax[1] = plot_pacf(dataset['Price'], ax=ax[1], lags=100)

plt.show()

Перевіримо ряд на стаціонарність.

In [ ]:
dickey_fuller_test(dataset['Price'])